# Homework


### Question 1

Read the data for  January and February 2023.

Yellow Taxi Trip Records

How many columns are there?



In [1]:
import os
from joblib import dump, load

import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

categorical = ['PULocationID', 'DOLocationID']

def save_model(model, filename):
    try:
        dump(model, filename)
        print("Model saved successfully.")
    except Exception as e:
        print("Error occurred while saving the model:", e)

def load_model(filename):
    try:
        model = load(filename)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print("Error occurred while loading the model:", e)
        return None

def get_target(input_df):
    minutes_duration = (
        (input_df['tpep_dropoff_datetime'] - input_df['tpep_pickup_datetime'])
        .dt
        .total_seconds()
        .div(60)
    )
    return minutes_duration

root_dir = '/srv/data'

[i for i in os.listdir(root_dir) if ('2023' in i and 'yellow' in i)]

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


['yellow_tripdata_2023-01.parquet', 'yellow_tripdata_2023-02.parquet']

In [2]:
filenames = ['yellow_tripdata_2023-01.parquet', 'yellow_tripdata_2023-02.parquet']

df_dict = {}
for month in ['01',]:# ['01', '02']:
    df_dict[month] = pd.read_parquet(os.path.join(root_dir, f'yellow_tripdata_2023-{month}.parquet'))
    print('num_rows: %d, num cols: %d ' % df_dict[month].shape)

df_dict['01'].head()

/usr/local/lib/python3.8/site-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (


num_rows: 3066766, num cols: 19 


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Question 2

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [3]:
cur_month = '01'

minutes_duration = get_target(df_dict[cur_month])

std_minutes_duration = (
    minutes_duration
    .std()
)

print(np.round(std_minutes_duration, 2))

42.59


## Question 3

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [4]:
outlier_filter = (minutes_duration >= 1) & (minutes_duration <= 60)
print('Left fraction: %.2f' % np.round((outlier_filter).value_counts(normalize=True).get(True), 2))

filtered_df = df_dict[cur_month][outlier_filter]
print(filtered_df.shape[0])

Left fraction: 0.98
3009173


## Question 4

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it

In [5]:

for col in categorical:
    filtered_df[col] = filtered_df[col].astype('str')
train_dicts = filtered_df[categorical].to_dict(orient='records')

dv = DictVectorizer()
dv.fit(train_dicts)
print('Transformation started')
X_train = dv.transform(train_dicts)

print('num cols: %d, num %d' % (X_train.shape[1], X_train.shape[0]))

/tmp/ipykernel_7288/1619250850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col] = filtered_df[col].astype('str')


Transformation started
num cols: 515


Warning: incorrect

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?



In [6]:
y_train = minutes_duration[outlier_filter]

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)

print('rmse %.5f' % rmse)

rmse 7.64926


In [7]:
save_model(lr, os.path.join(root_dir, 'lr.joblib'))
save_model(dv, os.path.join(root_dir, 'label_encoder.joblib'))

Model saved successfully.
Model saved successfully.


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [2]:
lr = load_model(os.path.join(root_dir, 'lr.joblib'))
dv = load_model(os.path.join(root_dir, 'label_encoder.joblib'))

Model loaded successfully.
Model loaded successfully.


In [3]:
month = '02'
valid_df = pd.read_parquet(os.path.join(root_dir, f'yellow_tripdata_2023-{month}.parquet'))

print('num rows %d' % valid_df.shape[0])

valid_df.head()

/usr/local/lib/python3.8/site-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (


num rows 2913955


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [4]:
valid_target = get_target(valid_df)

print(valid_target.mean())

16.01590961997241


Filter outliers (incorrect step, i think, but result doesn't match answers if you miss that step)

In [5]:
valid_outlier_filter = (valid_target >= 1) & (valid_target <= 60)
print('Left fraction: %.2f' % np.round((valid_outlier_filter).value_counts(normalize=True).get(True), 2))

filtered_valid_df = valid_df[valid_outlier_filter]
print(filtered_valid_df.shape[0])

Left fraction: 0.98
2855951


Categorical encoding

In [6]:
for col in categorical:
    valid_df[col] = valid_df[col].astype('str')
# test_dicts = valid_df[categorical].to_dict(orient='records')
test_dicts = filtered_valid_df[categorical].to_dict(orient='records')

print('Transformation started')
X_test = dv.transform(test_dicts)

X_test.shape

Transformation started


(2855951, 515)

Validation RMSE

In [7]:
# y_test = valid_target
y_test = valid_target[valid_outlier_filter]

y_pred = lr.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Validation RMSE %.4f' % rmse)

Validation RMSE 13.3227
